# Scraping Papers Data for Multiple Subjects from the OU Papers Web

### Import the Libraries again

In [11]:
# Import 'requests', a library to help retrieve web pages
import requests
#Import the csv library for outputting the table data
import csv
# BeautifulSoup can parse web page structure, much like a web browser does
from bs4 import BeautifulSoup

### Add the Names of all your Subjects

In [12]:
# Store the names of the subject codes you want to check in a variable - subjects
subjects = ("soci", "anth", "sowk", "geog", "gend", "crim")
print (subjects)

('soci', 'anth', 'sowk', 'geog', 'gend', 'crim')


### Create the Search URLS for all your Subjects

This is the second second section

In [13]:
# Use an 'accumulator pattern' to gather the variations of URL search including the different subjects
# make an empty list, then add the URLs to it one by one
subject_papers_search_urls = []

for item in subjects:
    url_ending = item + '&keywords=&period=&year=&distance=&lms=&submit=Search' 
## code for adding to the search end ?subjcode=*&papercode=musi&keywords=&period=&year=&distance=&lms=&submit=Search'
    subject_papers_search_urls.append('https://www.otago.ac.nz/courses/papers/index.html?subjcode=*&papercode=' + url_ending)

print(subject_papers_search_urls)

['https://www.otago.ac.nz/courses/papers/index.html?subjcode=*&papercode=soci&keywords=&period=&year=&distance=&lms=&submit=Search', 'https://www.otago.ac.nz/courses/papers/index.html?subjcode=*&papercode=anth&keywords=&period=&year=&distance=&lms=&submit=Search', 'https://www.otago.ac.nz/courses/papers/index.html?subjcode=*&papercode=sowk&keywords=&period=&year=&distance=&lms=&submit=Search', 'https://www.otago.ac.nz/courses/papers/index.html?subjcode=*&papercode=geog&keywords=&period=&year=&distance=&lms=&submit=Search', 'https://www.otago.ac.nz/courses/papers/index.html?subjcode=*&papercode=gend&keywords=&period=&year=&distance=&lms=&submit=Search', 'https://www.otago.ac.nz/courses/papers/index.html?subjcode=*&papercode=crim&keywords=&period=&year=&distance=&lms=&submit=Search']


In [14]:
#Create and open a file called Paper_Results_Test1.csv in append mode 'a' as a binary 'b'  
with open("CombinedPaper_Results_Test.csv", "w", encoding="utf-8", newline='\n') as fd:  
    fieldnames = [Paper_code, Year, Title, Points, Teaching_period]
    csvwriter = csv.writer(fd)
    csvwriter.writerow(["Paper code", "Year", "Title", "Points", "Teaching period"])

In [15]:
#Create a loop to process the successive pages
for subject_url in subject_papers_search_urls:
    
    #pull the data from the page
    r = requests.get(subject_url)
    # Print the status code and first 300 characters of the webpage text to check that it's working
    print(r.status_code)
    print(r.text[:300])
    
    #Make the Soup!
    soupextra = BeautifulSoup(r.text, 'lxml')
    
    #check that it's working
    #print (soupextra)

    #Pulling out the table data from the page and storing it in a variable
    #we can get away with with soup.table here because there's only one table on the page
    paper_results_rows = soupextra.find_all('tr')
    #print (paper_results_rows)
    
    #a loop to pull out all the relevant data - the square bracket 1: means miss out one row at the start
    for tr in paper_results_rows[1:]:
        td = tr.find_all('td') 
        row = [i.text for i in td]

        # This structure isolates each piece of row data item by its column in the table and converts it into a string.    
        Paper_code = str(td[0].get_text()) 
        Year = str(td[1].get_text())
        Title = str(td[2].get_text())
        Points = str(td[3].get_text())
        Teaching_period = str(td[4].get_text())    
    
        #Check that its working
        print (Paper_code, Year, Title, Points, Teaching_period)
    
        with open("CombinedPaper_Results_Test.csv", "a", encoding='UTF-8',newline='\n') as fd:  
            csvwriter = csv.writer(fd)
            csvwriter.writerow([Paper_code, Year, Title, Points, Teaching_period])

200
<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en-NZ"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8 lang="en-NZ""> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9" lang="en-NZ"> <![endif]-->
<!--[if gt IE 8]><!--> <html
SOCI101 2020 Sociology of New Zealand Society 18 points First Semester
SOCI102 2020 Cultural and Social Identities 18 points Second Semester
SOCI103 2020 Crime, Deviance and Social Transformation 18 points Second Semester
SOCI201 2020 Sociological Research in Practice 18 points First Semester
SOCI202 2020 Big Ideas in Sociology 18 points First Semester
SOCI203 2020 Young People and Society 18 points Second Semester
SOCI204 2020 Special Topic: Sociology of Health 18 points Second Semester
SOCI205 2020 Social Inequality 18 points Second Semester
SOCI207 2020 Families and Society 18 points Second Semester
SOCI208 2020 Environmental Sociology 18 points First Semester
SOCI213 2020 Concepts of th

200
<!DOCTYPE html>
<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en-NZ"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8 lang="en-NZ""> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9" lang="en-NZ"> <![endif]-->
<!--[if gt IE 8]><!--> <html
GEOG101 2020 Physical Geography 18 points First Semester
GEOG102 2020 Human Geography 18 points Second Semester
GEOG210 2020 Social Geography 18 points Second Semester
GEOG212 2020 Transformations in Developing Countries 18 points Not offered, expected to be offered in 2021
GEOG214 2020 Economic Geography of the Post-World War II Era 18 points First Semester
GEOG215 2020 Urban Geography 18 points Not offered, expected to be offered in 2021
GEOG216 2020 Resource Evaluation and Planning 18 points First Semester
GEOG219 2020 Special Topic in Human Geography 18 points Second Semester
GEOG228 2020 Uneven Development in Contemporary India 18 points First Semester
GEOG276 2020 Geographies of Conte

In [25]:
#